# Custom Data Generation for YOLOv7+ with VOCDataset and Training of YOLOv12
1. Take pictures
1. Prepare object images
1. Prepare environment
1. Organize the object images
3. Download VOCDataset for backing of object images
1. Overlay objects on backing images and generate YOLO labels
9000. Train model
1. Upload to Luxonis to convert to OpenVINO format
1. Export to Raspberry Pi

-------------------

TODO:
- test on Windows
- write program for editing and cropping object images (remove background: rembg 2.0.28) (crop images: https://pillow.readthedocs.io/en/stable/reference/Image.html#PIL.Image.Image.crop)
- test YOLO v12
- there are multiple errors/issues in 6 apparently (in each function!)
- same for 4 i guess...
- make sure 8 is correct
- idk how to do 9

-------------------

## 1. Take Pictures

When taking pictures of objects:
- TAKE MANY PHOTOS AT ALL POSSIBLE ANGLES (30-ish photos depending on how many angles we need)
- Good lighting and quality, of course
- Try to have the object be the only thing in frame (no extra objects that could possibly take away focus from the image)
- The object should be completely in frame and not cut off

Good example:

<img src="YOLO/markdownimages/baseimage.png" alt="goodexample-baseimage" width="300" height="300">


## 2. Prepare Object Images
Once you have your images head over to [remove.bg](https://remove.bg) and upload each image. Then open up each image without a background to GIMP or your photo editor of choice and crop the images until you are right at the edge of them down to the pixel. 
THIS IS TO BE REPLACED BY WHATEVER PROGRAM JULIA IS WRITING IDK (this file will do the editing process for you!!!)

Good example: (note how there is no background and it is cropped right to the edge of the object)

<img src="YOLO/markdownimages/editedimage.png" alt="goodexample-editedimage" width="300" height="300">


## 3. Prepare Environment
Run the below commands to make an environment (you will need to select the kernel as the venv after running) and clone the ultralytics github which contains yolov11, then cd into ultralytics:

THE VENV AND DIRECTORIES MAY ALREADY EXIST! if this is the case just let it be :)

In [8]:
!python3 -m venv .venv
!source .venv/bin/activate
%cd YOLO/
!git clone --progress --verbose https://github.com/ultralytics/ultralytics
%cd ultralytics/

# change directory into ultralytics so subsequent cmd line 
# operations are inside ultralytics
# create other folders that we will need later:

!mkdir ./objectImages
!mkdir ./ultralytics/Dataset
!mkdir ./ultralytics/Dataset/images
!mkdir ./ultralytics/Dataset/labels
!mkdir ./ultralytics/Dataset/images/test
!mkdir ./ultralytics/Dataset/images/train
!mkdir ./ultralytics/Dataset/labels/test
!mkdir ./ultralytics/Dataset/labels/train

[Errno 2] No such file or directory: 'YOLO/'
/home/lakemonsters2635/Desktop/MonsterVision5/YOLO/ultralytics
fatal: destination path 'ultralytics' already exists and is not an empty directory.
/home/lakemonsters2635/Desktop/MonsterVision5/YOLO/ultralytics/ultralytics


/home/lakemonsters2635/Desktop/MonsterVision5/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


mkdir: cannot create directory ‘./ultralytics/Dataset’: No such file or directory
mkdir: cannot create directory ‘./ultralytics/Dataset/images’: No such file or directory
mkdir: cannot create directory ‘./ultralytics/Dataset/labels’: No such file or directory
mkdir: cannot create directory ‘./ultralytics/Dataset/images/test’: No such file or directory
mkdir: cannot create directory ‘./ultralytics/Dataset/images/train’: No such file or directory
mkdir: cannot create directory ‘./ultralytics/Dataset/labels/test’: No such file or directory
mkdir: cannot create directory ‘./ultralytics/Dataset/labels/train’: No such file or directory


Check CUDA version if you have CUDA for pytorch installation (if you don't have CUDA then skip down to the second pip install segment):

In [9]:
!nvidia-smi

Wed Oct 22 21:00:49 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3090        Off |   00000000:02:00.0  On |                  N/A |
|  0%   41C    P8             36W /  370W |     559MiB /  24576MiB |     24%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Visit [pytorch.org/get-started/locally/](https://pytorch.org/get-started/locally/) to install the correct version of pytorch (only change the url part of the command and if you have a newer version of CUDA than pytorch has then install the latest pytorch version). This could take a while depending on your network speed.

IF YOU DO NOT HAVE CUDA SKIP TO THE NEXT CODE BLOCK

In [10]:
%pip install torch torchvision --index-url https://download.pytorch.org/whl/cu126

Looking in indexes: https://download.pytorch.org/whl/cu126
  Using cached https://download.pytorch.org/whl/cu126/torch-2.9.0%2Bcu126-cp310-cp310-manylinux_2_28_x86_64.whl (832.9 MB)
  Using cached https://download.pytorch.org/whl/cu126/torchvision-0.24.0%2Bcu126-cp310-cp310-manylinux_2_28_x86_64.whl (7.3 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 16.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.2/287.2 MB 10.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.3/322.3 MB 10.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 20.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 KB 12.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 

Install dependencies for yolov11 and all other ultralytics yolo versions:

In [ ]:
%pip install ultralytics tqdm>=4.41.0 pillow

If you restart the kernel make sure to come back to your path i think uhhhhh idk

In [ ]:
%cd ultralytics/ 
# should print /home/lakemonsters2635/Desktop/MonsterVision5/YOLO/ultralytics

## 4. Organize Object Images
Currently the file structure of important files should look like:
```
ultralytics
├── ultralytics
|     ├── Dataset
|     |     ├── images
|     |     └── labels
|     └── (folders with all of the yolo code and other things we will get to later)
├── objectImages
└── (more yolo files and folders)
```

Rename all of the images to `*class*_*number*`<br>
ex: `blue_1`, `cone_29`, `banana_0`

#### Take all of the images you just edited and put them in the folder named `objectImages`

Now edit the list below to contain all of your class names. We will need this later for generating the dataset and training the model.

In [ ]:
classNames = ["algae"] # Edit
import os
import json
CLASSES = {key: index for index, key in enumerate(classNames)} # DON'T EDIT
os.environ['CLASSES'] = json.dumps(CLASSES)
print(json.dumps(CLASSES))

## 5. Download VOCDataset for Backing of Object Images
We use the VOCDataset or Visualized Object Classes Dataset which is a dataset that contains many images with labels for training of pascal. We are just extracting the images from a few of the datasets for backing images for our yolo training images.

Download the 2007 and 2012 VOCDataset and put them in a separate directory (may take 10+ minutes depending on wifi) and extract the downloaded .tar archive files (should create folders `VOC2007` and `VOC2012` in `VOCdevkit`. If not then run again):

In [ ]:
# LINUX/MAC

!mkdir -p ./VOCdevkit 
# -p checks if the directory already exists before making
!wget -P ./VOCdevkit https://www.kaggle.com/api/v1/datasets/download/gopalbhattrai/pascal-voc-2012-dataset
!unzip ./VOCdevkit/pascal-voc-2012-dataset -d ./VOCdevkit/
!mkdir -p ./VOCdevkit/test_dataset/
!mkdir -p ./VOCdevkit/train_dataset/

In [ ]:
# WINDOWS

!mkdir -p ./VOCdevkit
# -p checks if the directory already exists before making
!curl -L -o ./VOCdevkit/pascal-voc-2012-dataset https://www.kaggle.com/api/v1/datasets/download/gopalbhattrai/pascal-voc-2012-dataset
!tar -xvf ./VOCdevkit/pascal-voc-2012-dataset.zip -C ./VOCdevkit/
!mkdir -p ./VOCdevkit/test_dataset/
!mkdir -p ./VOCdevkit/train_dataset/

Now we convert VOC dataset to YOLO-format.

In [ ]:
import os
for i in os.listdir('./VOCdevkit/VOC2012_test/VOC2012_test/JPEGImages'): # listdir wil return a list of strings of every file and folder name in the DIR
    os.rename('./VOCdevkit/VOC2012_test/VOC2012_test/JPEGImages/' + i, './VOCdevkit/test_dataset/' + i) # takes original path and appends each image file then moves it by "renaming" it
for i in os.listdir('./VOCdevkit/VOC2012_train_val/VOC2012_train_val/JPEGImages'):
    os.rename('./VOCdevkit/VOC2012_train_val/VOC2012_train_val/JPEGImages/' + i, './VOCdevkit/train_dataset/' + i)

Next, clean up your files:

In [ ]:
# LINUX/MAC

!rm ./VOCdevkit/pascal-voc-2012-dataset
!rm -rf ./VOCdevkit/VOC2012_test/
!rm -rf ./VOCdevkit/VOC2012_train_val/

In [ ]:
# WINDOWS

!del ./VOCdevkit/pascal-voc-2012-dataset
!rmdir ./VOCdevkit/VOC2012_test/
!rmdir ./VOCdevkit/VOC2012_train_val/



e









## 6. Overlay objects on backing images and generate YOLO labels

This script automatically generates synthetic training data for object detection models (such as YOLO) by compositing foreground object images onto random background images.

### Overview:
1. Randomly selects an object image (with transparent background) and a random background image.
2. Scales the object to a random percentage of the background’s size.
3. Pastes the scaled object onto a random position within the background.
4. Saves the resulting composite image to the dataset folder (e.g., ./ultralytics/Dataset/images/...).
5. Generates a corresponding YOLO-format label (.txt) containing the object’s class ID and bounding box coordinates
   (normalized x_center, y_center, width, height).
6. Designed for multiprocessing to efficiently generate large datasets.

### Functions:
- stackAndScaleImage(): Scales and pastes one image onto another using PIL.
- selectScaleAndCreateYoloLabels(): Combines object and background, computes YOLO label data.
- combineRandomImages(): Picks random images from directories and combines them.
- makeImage(): Creates and saves a single labeled synthetic training image.

In [ ]:
%%writefile ./dataGen.py 
# we write to a new file so we can do multiprocessing on a separate python file (we cant do multiprocessing in .ipynb)
from PIL import Image
import json
import random
import os
from tqdm import tqdm as progressBar
import threading
import multiprocessing
import concurrent.futures
import argparse



def stackAndScaleImage(objectImage, backgroundImage, scalePercent, position): # Takes a first PNG image, scales it down a certain percentage and pastes it on a second PNG image.

# objectImage: PIL image of object
# backgroundImage: PIL image of background
# scalePercent: Percentage to scale down the first image. (MAKE FLOAT INSTEAD OF DUMB STUFF)
# position: Tuple of (x, y) coordinates to paste the scaled image.

  objectImage = Image.open(objectImage)
  backgroundImage = Image.open(backgroundImage)

  # Scale down the first image
  width, height = objectImage.size # extract the width and height of object
  newWidth = int(width * scalePercent) # create a new width for object based on the scalePercent and makes it an int
  newHeight = int(height * scalePercent) # create a new height for object based on the scalePercent and makes it an int
  objectScaled = objectImage.resize((newWidth, newHeight)) # use the resize method of a PIL Image to scale object to the new_width and new_height
 
  # Scaled the hight to add more variation to the data
  width, height = objectScaled.size # Reset the width and height variables to be the new width and height of object after scaling
  randomHeightScale = random.uniform(0.8, 1.2) # Choose a random scalePercent between the minimum and maximum values.
  # randomHeightScale = 1.0
  newHeight = int(height * randomHeightScale) # Calculate the newHeight based on the random scalePercent above
  if newHeight > backgroundImage.height:
    newHeight = backgroundImage.height
    print("triggered", newHeight)
  objectScaled = objectScaled.resize((width, newHeight)) # Resize the image just like above
  
  backgroundImage.paste(objectScaled, position, objectScaled) # Second parameter makes it so that the pixels with no value meant to be clear stay clear and aren't black

  return (backgroundImage, newHeight) # Return the final stacked image




# SCALE PERCENTAGE IS HOW MUCH OF THE FRAME YOU WANT TO TAKE UP NOT HOW MUCH YOU WANT TO SCALE THE OBJECT IMAGE DOWN
def selectScaleAndCreateYoloLabels(objectPath, backgroundPath, minSizePercent, maxSizePercent, objectClassStr, CLASSES): 
  # Combines two images by pasting a scaled version of an object onto a background. SCALES BASED ON WIDTH

  # objectPath: Path to the object image.
  # backgroundPath: Path to the background image.
  # minSizePercent: Minimum percentage to scale down object's WIDTH.
  # maxSizePercent: Maximum percentage to scale down object's WIDTH.

  object = Image.open(objectPath) # Open the first image as object
  background = Image.open(backgroundPath) # Open the second image as background

  objectWidth, objectHeight = object.size # Extract the width and height of object as objectWidth and objectHeight
  backgroundWidth, backgroundHeight = background.size # Extract the width and height of background as backgroundWidth and backgroundHeight

  # Choose a random scalePercent between the minimum and maximum values provided as parameters
  # NEVER ABOVE 1.0(?)
  scalePercent = random.uniform(minSizePercent, maxSizePercent)

  # as background's width before applying this random scale percent (should really be same as shortest side?)
  # baseScale = backgroundWidth / objectWidth # The baseScale is the ratio of how big background's width is compared to object's width (ONLY WITH MEASUREMENTS)
  if backgroundWidth <= backgroundHeight: # scale based on shortest side of background
    baseScale = backgroundWidth/objectWidth
    used = "width shortest so base scale is based on width"
  else:
    baseScale = backgroundHeight/objectHeight
    used = "height shortest so base scale is based on height"

  # print({
  #   "backgroundWidth": backgroundWidth,
  #   "object.width": object.width,
  #   "objectWidth": objectWidth,
  #   "baseScale": baseScale,
  #   "scalePercent": scalePercent,
  #   "baseScale * scalePercent": baseScale * scalePercent,
  #   "object.width * scalePercent/100": object.width * scalePercent/100,
  #   "object.height * scalePercent/100": object.height * scalePercent/100,
  # })


  scalePercent = baseScale * scalePercent # fix the scalePercent to include the baseScale between the 2 images and be proportional

  # Choose a random position for object in background
  widthOfObjectAfterScaling = int(object.width * scalePercent)   # predict width of object after scaling so you can choose a random width in bounds of background
  heightOfObjectAfterScaling = int(object.height * scalePercent) # predict height of object after scaling so you can choose a random height in bounds of background
  x = random.randint(0, backgroundWidth - widthOfObjectAfterScaling)     # select random x position for object in background

  # ERROR IS HERE: The error is because background isn't tall enough to fit object even after scaling so the randint is trying to selced from 0 to a negative number
  y = random.randint(0, backgroundHeight - heightOfObjectAfterScaling)   # select random y position for object in background

  (combinedImage, finalHeight) = stackAndScaleImage(objectPath, backgroundPath, scalePercent, (x, y)) # Use the stack_scaled_images function to combine the two images

  # finalWidth, finalHeight = combinedImage.size # WRONG

  # Save the paste_parameters as a json object
  debugParameters = {
    "width_background": backgroundWidth,
    "height_background": backgroundHeight,
    "paste_x": x,
    "paste_y": y,
    "paste_width": widthOfObjectAfterScaling,
    "paste_height": finalHeight,
    "scale_type": used
  }
  # The position for object to be pasted onto background is randomly selected within the bounds of background

  # https://docs.cogniflow.ai/en/article/how-to-create-a-dataset-for-object-detection-using-the-yolo-labeling-format-1tahk19/
  # YOLO labeling parameters
  pasteParametersYolo = {
    "objectClassNum": CLASSES[objectClassStr],
    "x_center": (x + widthOfObjectAfterScaling/2.0) / backgroundWidth, # calculate what percentage of the width of background the center of scaled object will be
    "y_center": (y + finalHeight/2.0) / backgroundHeight, # calculate what percentage of the height of background the center of scaled object will be
    "width": widthOfObjectAfterScaling / backgroundWidth, # calculate what percentage of the width of background does scaled object take
    "height": finalHeight / backgroundHeight, # calculate what percentage of the height of background does scaled object take
  }

  return (combinedImage, debugParameters, pasteParametersYolo) # return the PIL image object after stacking, the parameters used for pasting, and the parameters used for pasting in a YOLO suitable notation





def combineRandomImages(directory1, directory2, minSizePercent, maxSizePercent, CLASSES):
  # Combines two random images from the two directories using the first function, returns a PIL Image object with the combined images

  #   directory1: Path to the first directory.
  #   directory2: Path to the second directory.
  #   minSizePercent: Minimum percentage to scale down the first image.
  #   maxSizePercent: Maximum percentage to scale down the first image.

  # Get all files in directories. If one isn't an image it could break
  objectFiles = os.listdir(directory1) # Get a list of all files in the first directory
  backgroundFiles = os.listdir(directory2) # Get a list of all files in the second directory

  imageChosen = random.choice(objectFiles)
  objectClass = imageChosen.split("_")[0]

  # Choose random images
  objectPath = os.path.join(directory1, imageChosen) # Choose a random file from the first directory
  backgroundPath = os.path.join(directory2, random.choice(backgroundFiles)) # Choose a random file from the second directory

  # Use the combine_images_james_xy function to combine the two images.
  combinedImage, debugParameters, pasteParametersYolo = selectScaleAndCreateYoloLabels(objectPath, backgroundPath, minSizePercent, maxSizePercent, objectClass, CLASSES)

  # print(pasteParametersYolo)

  return (combinedImage, debugParameters, pasteParametersYolo) # re-return all of the returns from the combine_images_james_xy function

# def makeImage(testOrTrain="test", numImages=10, minSizePercent=.05, maxSizePercent=.8, i=-1):
def makeImage(args):
  (testOrTrain, numImages, minSizePercent, maxSizePercent, CLASSES, i) = args
  if i == -1:
    raise ValueError("Invalid i Value")
  # Combine the images from directory1 (game object) with images from directory2 (backgrounds).
  combinedImage, debugParameters, pasteParametersYolo = combineRandomImages("./objectImages", "./VOCdevkit/"+testOrTrain, minSizePercent, maxSizePercent, CLASSES)
  # Figure out a file name based on the current iteration and type of dataset
  baseFilename = f"{testOrTrain}_{i:0{6}d}" # Max 100000 file names
  # Save the image to the specified folder based on type of data set and use the above created filename
  combinedImage.save('./ultralytics/Dataset/images/'+testOrTrain+'/'+baseFilename+'.png')

  # Open/create a text file with the same name as the image and add the paste_parameters_yolo to it
  with open('./ultralytics/Dataset/labels/'+testOrTrain+'/'+baseFilename+'.txt', "w") as f:
    yoloData = pasteParametersYolo
    if round(yoloData['x_center'],6) > 1 or round(yoloData['y_center'],6) > 1:
      print("ERROR at", i)
      print(yoloData)
      print(debugParameters)
    f.write(f"{yoloData['objectClassNum']} {round(yoloData['x_center'],6)} {round(yoloData['y_center'],6)} {round(yoloData['width'],6)} {round(yoloData['height'],6)}") # write data and round it to the correct decimal places (first digit in string is the class)



# Main function that ties together all of the other functions to make it work
def makeData(testOrTrain="test", numImages=10, minSizePercent=.05, maxSizePercent=.8):
  if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--classes", type=str)
    parser.add_argument("--testOrTrain", default="test", type=str)
    parser.add_argument("--numImages", default=2, type=int)
    parser.add_argument("--minSizePercent", default=.05, type=float)
    parser.add_argument("--maxSizePercent", default=.8, type=float)

  inputArgs = parser.parse_args()

  CLASSES = json.loads(inputArgs.classes.replace("'", '"'))
  testOrTrain = inputArgs.testOrTrain
  numImages = inputArgs.numImages
  minSizePercent = inputArgs.minSizePercent
  maxSizePercent = inputArgs.maxSizePercent

  testOrTrain = testOrTrain.lower()
  

  args = tuple([(testOrTrain, numImages, minSizePercent, maxSizePercent, CLASSES, i) for i in range(numImages)]) # creates a tuple of tuple
  
  with concurrent.futures.ProcessPoolExecutor() as executor:
    executor.map(makeImage, args)
    print("yippie!!! it works") # to make sure it works :)
  ##########
  # for i in os.listdir("./ultralytics/Dataset/labels/train"):
  #   with open("./ultralytics/Dataset/labels/train/"+i, 'rt') as f:
  #     vals = f.read()
  #     vals = vals.split(" ")
  #     vals = list(map(float, vals))
  #     if vals[1] > 1 or vals[2] > 1:
  #       print(i, vals[1], vals[2], vals[3], vals[4])


Now we generate the test and train image sets. You should pick a size percent that makes sense for the application. I suggest doing some tests by just generating 50 images and then seeing how it looks. Here are the lines to make a test and train dataset:

#### train code:

In [ ]:
# %%writefile ./dataGenTrain.py
# from dataGen import *
!python3 dataGen.py --classes="$CLASSES" --testOrTrain="train" --numImages=15000 --minSizePercent=.1 --maxSizePercent=.8
# args = tuple([("train", 15000, 0.10, 0.80, "$CLASSES", i) for i in range(15000)]) # creates a tuple of tuple
# with concurrent.futures.ProcessPoolExecutor() as executor:
#     executor.map(makeImage, args)

In [ ]:
# !python3 dataGenTrain.py

#### test code:

In [ ]:
# %%writefile ./dataGenTest.py
# from dataGen import *
!python3 dataGen.py --classes="$CLASSES" --testOrTrain="test" --numImages=7500 --minSizePercent=.1 --maxSizePercent=.9
# args = tuple([("test", 7500, 0.10, 0.90, "$CLASSES", i) for i in range(7500)]) # creates a tuple of tuple
# with concurrent.futures.ProcessPoolExecutor() as executor:
#     executor.map(makeImage, args)


In [ ]:
# !python3 dataGenTest.py

Now we create the dataset config file for training yolo but make sure to enumerate the classes at the bottom and pick names that are appropriate:

In [ ]:
%%writefile ./ultralytics/cfg/datasets/CUSTOM.yaml
# Ultralytics YOLO 🚀, AGPL-3.0 license
# Documentation: # Documentation: https://docs.ultralytics.com/datasets/detect/voc/
# Example usage: yolo train data=CUSTOM.yaml
# parent
# ├── ultralytics
# └── datasets
#     └── CUSTOM

# Train/val/test sets as 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]
path: ./Dataset
train: images/train # train images (relative to 'path')  16551 images
val: images/test # val images (relative to 'path')  4952 images
test: images/test # test images (optional)

# Set the number of classes
nc: 1 # EDIT TO REFLECT CORRECT VALUES

# Classes
names: # EDIT HERE
  0: algae

## 7. Train Model

Now that we have the dataset we can train the model. First we will run the checks command to make sure everything is good (it has all the libraries, the computer is able to train it, etc).

In [ ]:
!yolo checks

To confirm we have pyTorch available on our system (prints as a boolean)

In [ ]:
import torch
print(torch.cuda.is_available())

Now that everything is good we will run the help command to see how to use the yolo command.

In [ ]:
!yolo help

We are going to use the nano version of yolov11 because it is the smallest, fastest, and most lightweight version. We will specify the name of the custom data config file that we created above as well as yolov11n.pt for the nano version of the pretrained model. I've decided on 10 epochs for just a short retraining of the model. The imgsz is 640 which will resize all of the images we created to 640x640 for input into the model. This number can be changed for speed over accuracy if needed. I set the batch size to use 90% of the available GPU memory as well as set cache to true to improve speed.

In [ ]:
!yolo settings datasets_dir=ultralytics

imgsz is set to 512 because it must be a multiple of 32 and not greater than 530 to work with the camera with the OV... sensor

In [ ]:
import os
for i in os.listdir('./VOCdevkit/VOC2012_test/VOC2012_test/JPEGImages'): # listdir wil return a list of strings of every file and folder name in the DIR
    os.rename('./VOCdevkit/VOC2012_test/VOC2012_test/JPEGImages/' + i, './VOCdevkit/test_dataset/' + i) # takes original path and appends each image file then moves it by "renaming" it
for i in os.listdir('./VOCdevkit/VOC2012_train_val/VOC2012_train_val/JPEGImages'):
    os.rename('./VOCdevkit/VOC2012_train_val/VOC2012_train_val/JPEGImages/' + i, './VOCdevkit/train_dataset/' + i)

In [ ]:
!yolo detect train data="./ultralytics/cfg/datasets/CUSTOM.yaml" model="yolo11n.pt" epochs=2 imgsz=512 name="lakeMONSTER;)" batch=0.5 patience=2 cache=disk workers=8 device=0

## 8. Upload to Luxonis to convert to OpenVINO format

### NOTE: tools.luxonis.com is no longer maintained and is not compatible with versions past Yolov11 or depthAIv3. (How unfortunate that those are both versions we want to upgrade our code to...)

Navigate to [tools.luxonis.com](https://tools.luxonis.com) and select the following parameters:

- Yolo Version: `YOLOv11`
- `RVC2`
- Upload your .pt file (based on whats above i assume it would be named `lakeMONSTER;).pt` lol)
- Image shape: `512`
- Shaves: `6`
- Use OpenVINO: `True`

since i havent tested all this yet, im kinda guessing here, but it spits out a `lakeMONSTER;).blob` and a `lakeMONSTER.json` or something like that

put the blob and json in `~/MonsterVision5/models` and also use it in the next step i think

## 9. Export to Pi
something with https://wpilibpi.local:1181 idk tho

instructions below copied from `~/MonsterVision5/README.md`

### How to toggle the camera server
1. Open command prompt
2. `ssh pi@wpilibpi` or `ssh pi@wpilibpi.local`
3. Go to [wpilibpi.local webserver](http://wpilibpi.local/) and change it to writable
4. `sudo nano /boot/mv.json`
5. Change the `'showPreview'` key to `false` or `true` depending on whether or not you need it
6. Restart MonsterVision

How to Restart MonsterVision:
1. Go to [wpilibpi.local webserver](http://wpilibpi.local/) and go to Vision Status
2. Click the red Kill button

IF KILL/TERMINATE doesn't seem to work then you have to go into htop and SIGKILL the main MonsterVision4.5.py:
1. Open command prompt
2. `ssh pi@wpilibpi` or `ssh pi@wpilibpi.local`
3. Go to [the wpilibpi.local webserver](http://wpilibpi.local/) and change it to writable
4. `htop` in terminal to view all processes
5. Click on the `python3 ./MonsterVision4.5.py` (should be in green)
6. Type `fn+f9` and then '9' to execute the SIGKILL command to kill the process
7. Hit enter to execute the command


### How to change model used
1. Open command prompt
2. `ssh pi@wpilibpi` or `ssh pi@wpilibpi.local`
3. Go to [wpilibpi.local webserver](http://wpilibpi.local/) and change it to writable
4. Type `cd ./<path to MonsterVision>/models`
6. Type `sudo cp ./<desired model .json file> /boot/nn.json`
7. Type `mv ./<latest best.blob> ./<appropriate name for .blob given season>`
8. Type `sudo nano /boot/nn.json`
9. Add between lines 6 and 7 (6.5) `"blob": "<chosen appropriate name given season>", `

6 7?
